In [22]:
# Imports

import torch
import torchvision
from torch import nn
import logging

import tensorflow as tf
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [17]:
# Hyperparameters
nbEpochs = 50
train_batch_size = 100
test_batch_size = 100
train = "SVHN"
test = "MNIST"
dataPath = '/home/pmirallesr/eclipse-workspace/DRCN/data/'

**TEXT**

In [ ]:
class ImpulseDenoising:
    """Erase a pixel with probability p"""

    def __init__(self, p = 0.5):
        self.p = p

    def __call__(self, x):
        return x.numpy() * np.random.binomial(1, p, size=x.shape)


In [23]:
# Data Transforms
# Normalizations commented as torch already imports normalized data on 32x32

MNIST_Transforms = []
#MNIST_Transforms.append(torchvision.transforms.Resize((32,32)))
# MNIST_Mean = 0.1307
# MNIST_StDev = 0.3081
# MNIST_Transforms.append(torchvision.transforms.Normalize(MNIST_Mean, MNIST_StDev))



SVHN_Transforms = []
SVHN_Transforms.append(torchvision.transforms.Grayscale())
# SVHN_Mean = 0.4657
# SVHN_StDev = 0.2025
# SVHN_Transforms.append(torchvision.transforms.Normalize(SVHN_Mean, SVHN_StDev))

# Data Augmentation - Geometric Transformations
# 20º random rotation
# 20% random height and width shifts
dataAugmentTransforms = []
dataAugmentTransforms.append(torchvision.transforms.RandomAffine(degrees = 20, translate = (0.2, 0.2)))

# Denoising

dataAugmentTransforms.append(ImpulseDenoising())

# Geometric transformations
# Zero-masked noise
# Gaussian noise


In [7]:
train_MNIST_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.MNIST(dataPath, \
                train = True, download = True, \
                transform = torchvision.transforms.Compose\
                (MNIST_Transforms)), batch_size = train_batch_size, shuffle = True)

test_MNIST_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.MNIST(dataPath, \
                train = False, download = True, \
                transform = torchvision.transforms.Compose\
                (MNIST_Transforms)), batch_size = test_batch_size, shuffle = True)

#DataLoader has irregular behaviour, does not autom create an SVHN folder
train_SVHN_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.SVHN(dataPath + "SVHN/", \
                split = "train", download = True, \
                transform = torchvision.transforms.Compose\
                (SVHN_Transforms)), batch_size = train_batch_size, shuffle = True)

test_SVHN_loader = torch.utils.data.DataLoader \
                (torchvision.datasets.SVHN(dataPath + "SVHN/", \
                split = "test", download = True , \
                 transform = torchvision.transforms.Compose \
                (SVHN_Transforms)), batch_size = test_batch_size, shuffle = True)


Using downloaded and verified file: /home/pmirallesr/eclipse-workspace/DRCN/data/SVHN/train_32x32.mat
Using downloaded and verified file: /home/pmirallesr/eclipse-workspace/DRCN/data/SVHN/test_32x32.mat


In [8]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
# Models

# Move function to some utils module
def calcPoolOutputSize(inputSize, kernelSize):
    return inputSize/(kernelSize[0]*kernelSize[1])

       
class Encoder(nn.Module):
    """Encoder common to Autoencoder and labeller"""

    def __init__(self, encoder):
        """Initialize DomainRegressor."""
        super()
        self.encoder = encoder
        
        #Size Parameters
        
        conv1OutputSize = 100
        conv1KernelSize = 5
        
        maxPool1Size = (2,2)
        
        conv2OutputSize = 150
        conv2KernelSize = 5
        
        maxPool2Size = (2,2)
        
        conv3OutputSize = 200
        conv3KernelSize = 3
        
        # Placeholder ranges
        fc4OutputSize = range(300,1000,50)
        fc5OutputSize = range(300,1000,50)
        
        
        # Size Calculations
        conv1InputSize = IMGSIZE
        
        conv2InputSize = calcPoolOutputSize(conv1OutputSize,maxPool1Size)
        
        conv3InputSize = calcPoolOutputSize(conv2OutputSize,maxPool2Size)
        
        fc4InputSize = conv3OutputSize
        
        fc5InputSize = fc4OutputSize
        
        # Layers
        
        self.conv1 = nn.Conv2d(conv1InputSize, conv1OutputSize, conv1KernelSize)
        self.maxPool2D1 = nn.MaxPool2D(maxPool1Size)
        self.conv2 = nn.Conv2d(conv2InputSize, conv2OutputSize, conv2KernelSize)
        self.maxPool2D2 = nn.MaxPool2D(maxPool2Size)
        self.conv3 = nn.Conv2d(conv3InputSize, conv3OutputSize, conv3KernelSize)
        
        self.fc4 = nn.Linear(fc4InputSize, fc4OutputSize)
        self.fc5 = nn.Linear(fc5InputSize, fc5OutputSize)
        


    def forward(self, x):
        """Forward pass X and return probabilities of source and domain."""
        x = F.relu(self.conv1(x))
        x = F.relu(self.maxPool2D1(x))
        
        x = F.relu(self.conv2(x))
        x = F.relu(self.maxPool2D2(x))
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.maxPool2D3(x))
        
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        return x

class Labeller(nn.Module):
    """ The labeller part of the network is constituted by 
    the common Encoder plus a labelling fully connected layer"""
    def __init__(self, encoder):
        self.encoder = Encoder()
        fc3OutputSize = numLabels
        fc3InputSize = self.fc5.out_features
        self.fcOUT = nn.Linear(fc3InputSize, fc3OutputSize)  
        
    def forward(self, x):
        x = self.encoder(x)
        return F.relu(self.fcOUT(x))

class Autoencoder(nn.Module):
    """The autoencoder is constituted by the Encoder common to
    the labeller and itself, and a decoder part that is a mirror
    image of the Encoder
    
    Layers 6 and 7 are FC layers, layers 8 through 10 are (de)convolutional layers
    
    """

    def __init__(self, encoder):
        """Initialize DomainRegressor."""
        super()
        
        self.encoder = encoder
        #Size Parameters
        
        deconv8InputSize = 100
        deconv8KernelSize = 5
        
        upscale8Size = self.maxPool2D2.kernel_size
        
        deconv9InputSize = 150
        deconv9KernelSize = 5
        
        upscale9Size = self.maxPool2D1.kernel_size
        
        deconv10InputSize = 200
        deconv10KernelSize = 3
        
        # Placeholder ranges - substitute for actual numbers for usage
        fc7OutputSize = range(300,1000,50)
        fc6OutputSize = range(300,1000,50)
        
        
        # Size Calculations
        deconv8OutputSize = IMGSIZE
        
        #Is calcPoolSize still valid for Unspooling?
        
        deconv9OutputSize = calcPoolOutputSize(deconv8OutputSize,maxPool8Size)
        
        deconv10OutputSize = calcPoolOutputSize(deconv9OutputSize,maxPool9Size)
        
        conv8InputSize = fc7OutputSize
        
        fc7InputSize = fc6OutputSize
        
        # Layers
        
        self.fc7 = nn.Linear(fc7InputSize, fc7OutputSize)
        self.fc6 = nn.Linear(fc6InputSize, fc6OutputSize)
        
        self.deconv8 = nn.Conv2d(conv8InputSize, conv8OutputSize, conv8KernelSize)
        self.upsample8 = nn.upsample(scale_factor = 2, mode = "nearest")
        self.deconv9 = nn.Conv2d(conv9InputSize, conv9OutputSize, conv9KernelSize)
        self.upsample9 = nn.upsample(scale_factor = 2, mode = "nearest")
        self.deconv10 = nn.Conv2d(conv10InputSize, conv10OutputSize, conv10KernelSize)
        
        


    def forward(self, x):
        """Forward pass X and return probabilities of source and domain."""
        x = self.encoder(x)
        
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        
        x = F.relu(self.deconv8(x))
        x = F.relu(self.maxUnspool2D8(x))
        
        x = F.relu(self.deconv9(x))
        x = F.relu(self.maxUnspool2D9(x))
        
        x = self.deconv10(x)
        
       
        return x
    

In [18]:
encoder = Encoder()
labeller = Labeller(encoder)
autoencoder = Autoencoder(encoder)
labellerParams = list(encoder.parameters()) + list(labeller.parameters())
autoencoderParams = list(encoder.parameters()) + list(decoder.parameters())

NameError: name 'params' is not defined

In [ ]:
# controlPenalty = 0.4...0.7
optimizer = torch.optim.RMSprop(list(encoder.parameters, lr=1e-4, alpha=0.9, eps=1e-08, weight_decay=0.9, momentum=0, centered=False)

In [21]:
for epoch in range(nbEpochs):
    encoder.train()
    labeller.train()
    optimizer.zero_grad()
    for batch_id, (data, target) in enumerate(train_SVHN_loader):
        forward = labeller(data)
        loss = nn.CrossEntropyLoss()
        loss = loss(forward, target)
        tf.summary.scalar('Labeller Loss', loss, step=epoch)
        loss.backward()
        optimizer.step()
        
    for batch_id, (data, target) in enumerate(train_MNIST_loader):
        forward = autoencoder(data)
        loss = nn.MSELoss()
        loss = loss(forward, target)
        tf.summary.scalar('Autoencoder Loss', loss, step=epoch)
        loss.backward()
        optimizer.step()
    

SyntaxError: unexpected EOF while parsing (<ipython-input-21-b1c6a5b4afb9>, line 13)